# gathering information about github discussion

In [8]:
    import os, midgy
    from pandas import *
    from toolz.curried import *
    __import__("dotenv").load_dotenv()
    client = __import__("python_graphql_client").GraphqlClient(
        "https://api.github.com/graphql", dict(Authorization=F"token {os.environ['GITHUB_TOKEN']}")
    )
    import bs4
    Soup = partial(bs4.BeautifulSoup, features="html.parser")

In [9]:
%%
    frame = DataFrame(
        await client.execute_async(query :=
```graphql
query {
  repository(owner: "a11yhood", name: "community") {
    discussion(number: 1) {
      comments(first: 100) {
        nodes {
          id
          bodyHTML
        	reactions(first: 100) {
            nodes {
              user {
                login
              }
              content
            }
          }
          author {
            login
          }
        }
      }
    }
  }
}
```
    ))

frame = DataFrame(
 await client.execute_async(query :=
 
 query { 
 repository ( owner : "a11yhood" , name : "community" ) { 
 discussion ( number : 1 ) { 
 comments ( first : 100 ) { 
 nodes { 
 id 
 bodyHTML 
 reactions ( first : 100 ) { 
 nodes { 
 user { 
 login 
 } 
 content 
 } 
 } 
 author { 
 login 
 } 
 } 
 } 
 } 
 } 
 } 
 
 
 ))

In [10]:
(
    df :=    
     frame.data
    .apply(Series)
    .discussion
    .apply(Series)
    .comments
    .apply(Series)
    .nodes
    .explode()
    .apply(Series)
    .set_index("id")
)

reactions = df.reactions.apply(Series).nodes.explode().dropna().apply(Series)

df = df.drop(columns="reactions")

df.author = df.author.apply(get("login"))

(
    links:= df.bodyHTML
    .apply(Soup)
    .apply(operator.methodcaller("select", "a"))
    .explode()
    .dropna()
)

a = links.iloc[0]

parts = juxt(operator.methodcaller("get_text"), compose_left(
    operator.attrgetter("attrs"), get("href")
))

df = links.apply(
    compose_left(parts, partial(zip, "title href".split()), dict, Series)
).join(df)

In [11]:
links = df["author title href".split()].set_index("href")

In [12]:
import urllib

In [13]:
links.index.to_series().apply(
    compose_left(urllib.parse.urlparse, operator.attrgetter("netloc"))
).value_counts()

href
github.com           3
opendesign.guide     1
see3d.org            1
vis.csail.mit.edu    1
arxiv.org            1
dl.acm.org           1
www.youtube.com      1
Name: count, dtype: int64